In [ ]:
! git clone https://github.com/iolkhovsky/retinanet_torch
%cd retinanet_torch/
! pwd
! ls

Cloning into 'retinanet_torch'...
remote: Enumerating objects: 334, done.
remote: Counting objects: 100% (334/334), done.
remote: Compressing objects: 100% (309/309), done.
remote: Total 334 (delta 203), reused 150 (delta 19), pack-reused 0
Receiving objects: 100% (334/334), 13.87 MiB | 21.82 MiB/s, done.
Resolving deltas: 100% (203/203), done.
/content/retinanet_torch
/content/retinanet_torch
configs  docs	 requirements.txt  test_images	train_lightning.py  utils
dataset  models  sandbox.ipynb	   tests	train.py


In [ ]:
! git pull origin develop

From https://github.com/iolkhovsky/retinanet_torch
 * branch            develop    -> FETCH_HEAD
Already up to date.


In [ ]:
%%capture
! pip install git+https://github.com/PytorchLightning/pytorch-lightning.git@master --upgrade

In [ ]:
!pip install tensorboard --upgrade

Requirement already up-to-date: tensorboard in /usr/local/lib/python3.7/dist-packages (2.4.1)


In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
import sys
from os.path import join
from pathlib import Path

parent_dir = str(Path().parent.absolute())
sys.path.insert(0, parent_dir)
print(sys.path)

['/content/retinanet_torch', '', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']


In [7]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import datetime
from os.path import join
from os import makedirs

session_timestamp = str(datetime.datetime.now())
session_timestamp = session_timestamp.replace(" ", "").replace(":", "-").replace(".", "-")
session_storage = join("/content/drive/MyDrive", "colab_workspace", 
                       session_timestamp)
makedirs(session_storage)
print(f"Session storage has bee created: {session_storage}")

checkpoints_root = join(session_storage, "checkpoints")
makedirs(checkpoints_root)
print(f"Checkpoints storage has bee created: {checkpoints_root}")
logs_root = join(session_storage, "logs")
makedirs(logs_root)
print(f"Tensorboard logs storage has bee created: {logs_root}")

Session storage has bee created: /content/drive/MyDrive/colab_workspace/2021-03-1018-24-06-343863
Checkpoints storage has bee created: /content/drive/MyDrive/colab_workspace/2021-03-1018-24-06-343863/checkpoints
Tensorboard logs storage has bee created: /content/drive/MyDrive/colab_workspace/2021-03-1018-24-06-343863/logs


In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.tensorboard import SummaryWriter

from models.ssd_mobilenet_v2 import SSDLightning

model = SSDLightning(classes_cnt=21, 
                     tboard_writer=writer, 
                     train_batch=32, 
                     val_batch=64)
print(f"Model has been built {model}")

writer = SummaryWriter(log_dir=logs_root)
print(f"Writer initialized at {logs_root}")

checkpoint_callback = ModelCheckpoint(
    filepath=join(checkpoints_root, "retinanet_voc.ckpt"),
    save_best_only=True,
    verbose=True,
    monitor='val_loss',
    mode='min'
)
print(f"Checkpoints storage: {checkpoints_root}")

trainer = pl.Trainer(max_epochs=20,
                     limit_val_batches=1,
                     gpus=1,
                     val_check_interval=50,
                     check_val_every_n_epoch=1,
                     checkpoint_callback=checkpoint_callback)

trainer.fit(model)


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params
-------------------------------------------------
0 | ssd       | SSDMobilenet2      | 31.0 M
1 | box_coder | FasterRCNNBoxCoder | 0     
2 | criterion | RetinaNetLoss      | 0     
-------------------------------------------------
31.0 M    Trainable params
0         Non-trainable params
31.0 M    Total params
124.076   Total estimated model params size (MB)



Extracting ../data/VOCtrainval_11-May-2012.tar to ../data
Using downloaded and verified file: ../data/VOCtrainval_11-May-2012.tar
Extracting ../data/VOCtrainval_11-May-2012.tar to ../data


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:51: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


KeyboardInterrupt: ignored